# 1 How many records are in the multi-FASTA file?

In [118]:
from Bio import SeqIO


In [94]:
def count_records(filename):
    record_count = 0
    with open(filename, 'r') as fasta_file:
        for line in fasta_file:
            if line.startswith('>'):
                record_count += 1
    return record_count

filename = "dna2.fasta"
num_records = count_records(filename)
print("Number of records in the file:", num_records)


Number of records in the file: 18


# 2 What is the length of the longest sequence in the file?

In [95]:
def get_sequence_lengths(filename):
    seq_lengths = {}
    with open(filename, 'r') as fasta_file:
        sequence_id = ''
        sequence = ''
        for line in fasta_file:
            if line.startswith('>'):
                if sequence_id:
                    seq_lengths[sequence_id] = len(sequence)
                sequence_id = line.strip()[1:]
                sequence = ''
            else:
                sequence += line.strip()
        seq_lengths[sequence_id] = len(sequence)
    return seq_lengths

filename = "dna2.fasta"
seq_lengths = get_sequence_lengths(filename)

longest_seq_length = max(seq_lengths.values())
longest_seqs = [seq_id for seq_id, seq_len in seq_lengths.items() if seq_len == longest_seq_length]

print("Length of the longest sequence:", longest_seq_length)
print("Identifiers of the longest sequences:", longest_seqs)


Length of the longest sequence: 4894
Identifiers of the longest sequences: ['gi|142022655|gb|EQ086233.1|255 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence']


In [96]:
def shortest_sequence_length(filename):
    shortest_length = float('inf')
    with open(filename, 'r') as fasta_file:
        current_sequence_length = 0
        for line in fasta_file:
            if line.startswith('>'):
                # We've reached a new sequence
                shortest_length = min(shortest_length, current_sequence_length)
                current_sequence_length = 0
            else:
                # We're in the middle of a sequence
                current_sequence_length += len(line.strip())
        # Don't forget to check the length of the last sequence in the file
        shortest_length = min(shortest_length, current_sequence_length)
    return shortest_length


# 3 What is the length of the shortest sequence in the file?

In [100]:
def get_seq_lengths(filename):
    seq_lengths = []
    with open(filename, 'r') as fasta_file:
        seq = ''
        for line in fasta_file:
            if line.startswith('>'):
                if seq:
                    seq_lengths.append(len(seq))
                    seq = ''
            else:
                seq += line.strip()
        if seq:
            seq_lengths.append(len(seq))
    return min(seq_lengths)

print('The shortest sequence in the file has a length of:', get_seq_lengths('dna2.fasta'))


The shortest sequence in the file has a length of: 115


# 10 Which one of the following repeats of length 7 has a maximum number of occurrences?

In [111]:
from collections import Counter
import re

# Read the sequences from the file
with open('dna2.fasta') as file:
    sequences = {}
    for line in file:
        line = line.rstrip()
        if line.startswith('>'):
            seq_name = line[1:]
            sequences[seq_name] = ''
        else:
            sequences[seq_name] += line

# Concatenate all the sequences into a single string
seq = ''.join(sequences.values())

# Find all repeats of length 7
repeats = re.findall(r'(?=(\w{7}))', seq)

# Count the occurrences of each repeat
repeat_counts = Counter(repeats)

# Find the repeat with the maximum number of occurrences
max_repeat = repeat_counts.most_common(1)[0][0]

# Print the result
print(f"The repeat of length 7 that occurs the maximum number of times is '{max_repeat}' with a count of {repeat_counts[max_repeat]}.")


The repeat of length 7 that occurs the maximum number of times is 'CGCCGCG' with a count of 63.


# 4 What is the length of the longest ORF appearing in reading frame 2 of any of the sequences?

In [116]:
def find_longest_orf_start_rf2(filename):
    longest_orf = ""
    with open(filename) as f:
        for seq_record in SeqIO.parse(f, "fasta"):
            seq = str(seq_record.seq)
            for i in range(1, 4):
                if i == 2:  # check only for reading frame 2
                    rev_seq = seq[::-1].translate(str.maketrans("ATGC", "TACG"))
                    for strand, nuc in [(+1, seq), (-1, rev_seq)]:
                        for frame in range(3):
                            length = 3 * ((len(seq) - frame) // 3)
                            for j in range(frame, length, 3):
                                codon = nuc[j : j + 3]
                                if codon == "ATG":
                                    codon_start = j
                                    aa_seq = ""
                                    for k in range(j, length, 3):
                                        codon = nuc[k : k + 3]
                                        aa = codon_table.get(codon, "")
                                        if aa == "":
                                            break
                                        aa_seq += aa
                                    if len(aa_seq) > len(longest_orf):
                                        longest_orf = aa_seq
    return len(longest_orf)


In [117]:
filename = "dna2.fasta"
longest_orf_length = find_longest_orf_start_rf2(filename)
print(f"The length of the longest ORF in reading frame 2 is {longest_orf_length}")


The length of the longest ORF in reading frame 2 is 1625


# 5 What is the starting position of the longest ORF in reading frame 3 in any of the sequences? The position should indicate the character number where the ORF begins. For instance, the following ORF:

-> sequence1

-ATGCCCTAG

-starts at position 1.

In [119]:
filename = "dna2.fasta"
frame = 3

def find_longest_orf_start(filename, frame):
    longest_orf_start = 0
    longest_orf_len = 0
    with open(filename) as f:
        for seq_record in SeqIO.parse(f, "fasta"):
            seq = str(seq_record.seq)
            for i in range(frame - 1, len(seq) - 2, 3):
                codon = seq[i:i+3]
                if codon == "ATG":
                    orf_len = 3
                    end_pos = i + 3
                    while end_pos < len(seq) - 2:
                        next_codon = seq[end_pos:end_pos+3]
                        if next_codon in ["TAA", "TAG", "TGA"]:
                            break
                        orf_len += 3
                        end_pos += 3
                    if orf_len > longest_orf_len:
                        longest_orf_len = orf_len
                        longest_orf_start = i + 1
    return longest_orf_start

longest_orf_start = find_longest_orf_start(filename, frame)
print(f"The starting position of the longest ORF in reading frame {frame} is {longest_orf_start}")


The starting position of the longest ORF in reading frame 3 is 636


 # 9 Find all repeats of length 12 in the input file. Let's use Max to specify the number of copies

of the most frequent repeat of length 12.  How many different 12-base sequences 

occur Max times?

In [121]:
def find_max_repeats(filename, length):
    # Read in the sequences from the file
    sequences = read_fasta(filename)
    # Create an empty dictionary to store the counts of each k-mer
    kmer_counts = {}
    # Iterate over each sequence
    for seq in sequences.values():
        # Iterate over each k-mer of length k
        for i in range(len(seq)-length+1):
            kmer = seq[i:i+length]
            if kmer in kmer_counts:
                kmer_counts[kmer] += 1
            else:
                kmer_counts[kmer] = 1
    # Find the maximum count of any k-mer
    max_count = max(kmer_counts.values())
    # Find all k-mers with count equal to max_count
    max_kmers = [kmer for kmer, count in kmer_counts.items() if count == max_count]
    return len(max_kmers), max_count


In [123]:
def read_fasta(filename):
    """Reads a multi-FASTA file and returns a dictionary of sequence IDs and sequences"""
    sequences = {}
    with open(filename, "r") as fasta_file:
        seq_id = None
        seq = ""
        for line in fasta_file:
            if line.startswith(">"):
                if seq_id is not None:
                    sequences[seq_id] = seq
                seq_id = line.strip()[1:]
                seq = ""
            else:
                seq += line.strip()
        if seq_id is not None:
            sequences[seq_id] = seq
    return sequences


In [124]:
filename = "dna2.fasta"
length = 12
num_kmers, max_count = find_max_repeats(filename, length)
print(f"There are {num_kmers} different {length}-base sequences that occur {max_count} times.")


There are 4 different 12-base sequences that occur 10 times.


# 8 Find the most frequently occurring repeat of length 6 in all sequences. How many times does it occur in all?

In [125]:
from collections import Counter
from Bio import SeqIO

def find_most_common_kmer(filename, k):
    kmers = []
    with open(filename) as f:
        for seq_record in SeqIO.parse(f, "fasta"):
            seq = str(seq_record.seq)
            for i in range(len(seq) - k + 1):
                kmer = seq[i:i+k]
                kmers.append(kmer)
    kmer_counts = Counter(kmers)
    most_common_kmer, count = kmer_counts.most_common(1)[0]
    return most_common_kmer, count

filename = "dna2.fasta"
k = 6
most_common_6mer, count = find_most_common_kmer(filename, k)
print(f"The most frequent {k}-mer is {most_common_6mer}, which occurs {count} times.")


The most frequent 6-mer is GCGCGC, which occurs 153 times.


# 7 What is the length of the longest forward ORF that appears in the sequence with the identifier  gi|142022655|gb|EQ086233.1|16?

In [126]:
# Define a function to find the longest forward ORF in a DNA sequence
def longest_orf(seq):
    # Import the ORF finder from the BioPython library
    from Bio.Seq import Seq
    from Bio.SeqRecord import SeqRecord
    from Bio.Alphabet import generic_dna
    from Bio.SeqFeature import SeqFeature, FeatureLocation
    from Bio import SeqIO
    from Bio.SeqUtils import six_frame_translations

    # Translate the sequence in all six reading frames
    translations = six_frame_translations(seq)

    # Find the longest ORF in each translation
    orfs = []
    for trans in translations:
        for frame, orf in trans.items():
            if len(orf) > 0:
                orfs.append(SeqRecord(Seq(orf, generic_dna)))

    # Sort the ORFs by length, in descending order
    orfs.sort(key=lambda x: len(x), reverse=True)

    # Return the length of the longest ORF
    return len(orfs[0])

# Read in the sequences from the file
filename = "dna2.fasta"
sequences = read_fasta(filename)

# Extract the DNA sequence for the specific identifier
seq = sequences["gi|142022655|gb|EQ086233.1|16"]

# Find the length of the longest forward ORF in the sequence
length = longest_orf(seq)

print(f"The length of the longest forward ORF in the sequence with identifier 'gi|142022655|gb|EQ086233.1|16' is {length}.")


KeyError: 'gi|142022655|gb|EQ086233.1|16'

# Question 6
What is the length of the longest ORF appearing in any sequence and in any forward reading frame?

In [128]:
filename = "dna2.fasta"

# Read in the sequences from the file
sequences = read_fasta(filename)

# Find the length of the longest ORF in any sequence and any forward reading frame
max_orf_length = 0
for seq_id, seq in sequences.items():
    # Consider all six possible reading frames
    for frame in range(3):
        # Extract the subsequence in this frame
        subseq = seq[frame:]
        # Find the length of the longest ORF in this subsequence
        orf_length = longest_orf(subseq)
        # Update the max ORF length if necessary
        if orf_length > max_orf_length:
            max_orf_length = orf_length

print("The length of the longest ORF in any sequence and any forward reading frame is:", max_orf_length)


ImportError: Bio.Alphabet has been removed from Biopython. In many cases, the alphabet can simply be ignored and removed from scripts. In a few cases, you may need to specify the ``molecule_type`` as an annotation on a SeqRecord for your script to work correctly. Please see https://biopython.org/wiki/Alphabet for more information.